In [1]:
import re
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Pre Dig FileName Xref 20230807 (002).csv')

In [3]:
#Create Columns
df['Polygon'] = pd.NA
df['Project Number'] = pd.NA
df['Work Order'] = pd.NA

In [4]:
def extract_project_number(filename):
    # Define the regular expression pattern to search for "RO####-###" in the 'FILENAME'.
    pattern = r'R0\d{4}-\d{3}'
    
    # Use the findall function to extract all occurrences of the pattern in the 'FILENAME'.
    matches = re.findall(pattern, filename)
    
    # If there are matches, return the first occurrence (assuming there's only one per filename).
    # If there are no matches, return None.
    return matches[0] if matches else None


In [5]:
def extract_work_order(filename):
    # Define the regular expression pattern to search for 'E0' followed by the next 6 characters.
    pattern = r'E0.{6}'
    
    # Use the findall function to extract all occurrences of the pattern in the 'FILENAME'.
    matches = re.findall(pattern, filename)
    
    # If there are matches, return the first occurrence (E0 followed by 6 characters).
    # If there are no matches, return None.
    return matches[0] if matches else None

In [6]:
def extract_polygon(filename):
    # Define the first regular expression pattern to search for the polygon in the 'FILENAME'.
    # The pattern captures everything up to 'CAB' followed by an optional hyphen and a number.
    pattern1 = r'^(.*?CAB(?:\s?-?\s?\d+))'
    
    # Use the findall function to extract all occurrences of the first pattern in the 'FILENAME'.
    matches1 = re.findall(pattern1, filename)
    
    if matches1:
        # If there are matches for the first pattern, return the first occurrence.
        return matches1[0].strip()
    else:
        # Define the second regular expression pattern to search for the polygon in the 'FILENAME'.
        # The second pattern captures everything up to "R0" without including "R0".
        pattern2 = r'^(.*?)(?:\sR0|$)'
        
        # Use the findall function to extract all occurrences of the second pattern in the 'FILENAME'.
        matches2 = re.findall(pattern2, filename)
        
        if matches2:
            # If there are matches for the second pattern, check if Polygon is the original string.
            # If Polygon is the original string, apply a third pattern to capture everything between two periods.
            if filename == matches2[0].strip():
                # Define the third regular expression pattern to capture everything between two periods.
                pattern3 = r'^(?:[^.]*\.)(.*?)(?:\.[^.]*|$)'
                
                # Use the findall function to extract all occurrences of the third pattern in the 'FILENAME'.
                matches3 = re.findall(pattern3, filename)
                
                # If there are matches for the third pattern, return the first occurrence.
                # If there are no matches for the third pattern, return None.
                return matches3[0].strip() if matches3 else None
            else:
                # If Polygon is not the original string, return whatever was captured by the second pattern.
                return matches2[0].strip()
        else:
            # If there are no matches for the second pattern, check for the fourth pattern.
            # The fourth pattern captures everything up to "UNVERIFIED ADDRESS LIST".
            pattern4 = r'^(.*?)(?:\s-?\s?UNVERIFIED ADDRESS LIST|$)'
            
            # Use the findall function to extract all occurrences of the fourth pattern in the 'FILENAME'.
            matches4 = re.findall(pattern4, filename)
            
            # If there are matches for the fourth pattern, return the first occurrence.
            # If there are no matches for any of the patterns, return None.
            return matches4[0].strip() if matches4 else None


In [7]:
def extract_polygon_2nd_try(filename):
    # Define the first regular expression pattern to capture everything from the beginning of the string
    # up until '(E0' (excluding it).
    pattern1 = r'^(.*?)(?:\(E0|$)'

    # Define the second regular expression pattern to capture everything from the beginning of the string
    # up until 'UNVERIFIED ADDRESSES.xls' or '- .xls' (excluding them).
    pattern2 = r'^(.*?)(?:UNVERIFIED ADDRESSES.xls|- \.xls|$)'

    # Use regex to extract the desired substring from 'filename'
    match1 = re.match(pattern1, filename)
    match2 = re.match(pattern2, filename)

    # Check if the regex matched for the first pattern, if not, try the second pattern
    if match1:
        return match1.group(1).strip()
    elif match2:
        return match2.group(1).strip()
    return None

In [8]:
def remove_unverified_addresses(filename):
    if filename is None:
        return None
    
    # Define the list of substrings to check and remove
    substrings_to_remove = ['UNVERIFIED ADDRESSES.xls', '- .xls', '.xls']
    
    # Check if any of the substrings is present in the 'filename'
    for substring in substrings_to_remove:
        if substring in filename:
            # Remove the substring from the 'filename' value
            filename = filename.replace(substring, '').strip()

    return filename

In [9]:
# Function to update 'Polygon' column for substrings
def update_polygon_column(row):
    row['Polygon'] = remove_unverified_addresses(row['Polygon'])
    return row

In [10]:
def process_polygon(row):
    if row['Polygon'] in ['xls', 'xlsx']:
        row['Polygon'] = extract_polygon_2nd_try(row['FILENAME'])
    return row

In [11]:
def update_polygon(row):
    if row['Project Number'] and row['Polygon']:
        if row['Project Number'] in row['Polygon']:
            row['Polygon'] = row['Polygon'].replace(row['Project Number'], '').strip().strip('.')
    return row

In [12]:
def update_polygon_with_regex(row):
    # Get the 'FILENAME' from the row
    filename = row['FILENAME']
    
    # Define the regular expression pattern to capture everything up until the first dot '.'
    pattern = r'^(.*?)\.'
    
    # Use regex to extract the desired substring from 'FILENAME'
    match = re.match(pattern, filename)
    
    # If the regex matched, update the 'Polygon' column with the extracted substring
    if match:
        row['Polygon'] = match.group(1).strip()
    
    return row

In [13]:
def update_polygon_with_unverified(row):
    if row['Polygon']:
        # Remove 'UNVERIFIED ADDRESS LIST' along with '-' from the 'Polygon' column
        row['Polygon'] = row['Polygon'].replace('UNVERIFIED ADDRESS LIST -', '').strip()
        row['Polygon'] = row['Polygon'].replace('UNVERIFIED ADDRESS LIST', '').strip()
    return row

In [14]:
def update_polygon_remove_copy_of(row):
    if row['FILENAME']:
        # Check if the 'Polygon' column contains the substring 'Copy of'
        if 'Copy of' in row['FILENAME']:
            # Remove 'Copy of' from the 'Polygon' column
            row['FILENAME'] = row['FILENAME'].replace('Copy of', '').strip()
    return row

In [15]:
def remove_project_number_from_filename(row):
    # Get the 'Project Number' from the row
    project_number = row['Project Number']
    
    # Check if the 'Project Number' is not NaN
    if pd.notna(project_number):
        # Get the 'FILENAME' from the row
        filename = row['FILENAME']
        
        # Remove the 'Project Number' from the 'FILENAME' if it exists
        row['FILENAME'] = filename.replace(project_number, '').strip()
    
    return row

In [16]:
def clean_polygon_values(polygon):
    if polygon is None:
        return None
    
    # Define the characters to be removed
    characters_to_remove = ['(', ')', '-', 'UNVERIFIED ADDRESSES', 'unverified ADDRESS LIST']
    
    # Remove the specified characters from the 'polygon' value
    for char in characters_to_remove:
        polygon = polygon.replace(char, '').strip()
    
    return polygon

In [17]:
def update_polygon_column_clean(row):
    row['Polygon'] = clean_polygon_values(row['Polygon'])
    return row


In [18]:
def remove_work_order_from_polygon(row):
    work_order = row['Work Order']
    polygon = row['Polygon']

    if work_order is not None and polygon is not None:
        # Check if the 'Work Order' value exists as a substring in the 'Polygon' value
        if work_order in polygon:
            # Remove the 'Work Order' value from the 'Polygon' value
            polygon = polygon.replace(work_order, '').strip()
            row['Polygon'] = polygon

    return row

In [19]:
def update_polygon_LCC_Edge(row):
    polygon = row['Polygon']
    filename = row['FILENAME']
    
    # Check if both polygon and filename are not None
    if polygon is not None and filename is not None:
        # Check if the substring '_LCC' exists in the 'Polygon' value
        if '_LCC' in polygon and filename.startswith('R0'):
            # Define the regex pattern to extract the desired substring
            pattern = r'(?<=^.{10})(.*?)(?=\.)'
            
            # Use the regex pattern to extract the substring from the 'FILENAME'
            extracted_value = re.search(pattern, filename)
            
            # Check if the extracted value is not None before accessing the group
            if extracted_value is not None:
                extracted_value = extracted_value.group(0).strip()
                row['Polygon'] = extracted_value
    
    return row

In [20]:
df = df.apply(update_polygon_remove_copy_of, axis=1)
# Now, let's apply the 'extract_project_number' function to the 'FILENAME' column in the DataFrame 'df' to create the new 'Project Number' column.
df['Project Number'] = df['FILENAME'].apply(extract_project_number)
# Now, let's apply the 'extract_work_order' function to the 'FILENAME' column in the DataFrame 'df' to create the new 'Work Order' column.
df['Work Order'] = df['FILENAME'].apply(extract_work_order)
# Now, let's apply the 'extract_polygon' function to the 'FILENAME' column in the DataFrame 'df' to create the new 'Polygon' column.
df['Polygon'] = df['FILENAME'].apply(extract_polygon)
df = df.apply(process_polygon, axis=1)
df = df.apply(update_polygon_with_unverified, axis=1)
df = df.apply(update_polygon, axis=1)
df = df.apply(update_polygon_column, axis=1)
df = df.apply(update_polygon_column_clean, axis=1)
df = df.apply(remove_work_order_from_polygon, axis=1)
for index, row in df.iterrows():
    df.loc[index] = update_polygon_LCC_Edge(row)
# Print the updated DataFrame
print(df)

                                              FILENAME  \
0    R01951-038.PORTSMOUTH RT 7 LCP #3A.(PR7LCP3AF)...   
1    VIRGINIA BEACH RT 30 LCP 5 R01952-091#5.(2205C...   
2    VIRGINIA BEACH RT 35 LCP 2  R01952-115#2.(2468...   
3    R01952-041.VIRGINIA BEACH RT 13 LCP #5 CAB 1.(...   
4    VIRGINIA BEACH RT 30 LCP 1 R01952-087#1.(2320S...   
..                                                 ...   
499  UNVERIFIED ADDRESS LIST - WINSTON SALEM 53 CAB...   
500  UNVERIFIED ADDRESS LIST - R01810-014 - E019DR2...   
501  UNVERIFIED ADDRESS LIST - R01810-006 - E026DR2...   
502      TROLLINGER ST (E032RM22) UNVERIFIED ADDRESSES   
503  UNVERIFIED ADDRESS LIST - R01810-001 - E013JT2...   

                               Polygon Project Number Work Order  
0              PORTSMOUTH RT 7 LCP #3A     R01951-038       None  
1           VIRGINIA BEACH RT 30 LCP 5     R01952-091       None  
2           VIRGINIA BEACH RT 35 LCP 2     R01952-115       None  
3    VIRGINIA BEACH RT 13 LCP #5 CA

In [26]:
# Create a copy of the DataFrame
df_remaining = df.copy()
# Drop rows where 'Polygon' column doesn't have parenthesis
#df_remaining = df_remaining[df_remaining['Polygon'].str.contains(r'\(.*\)')]
print(df_remaining)

df_remaining = df_remaining.apply(remove_project_number_from_filename, axis=1)
df_remaining = df_remaining.apply(update_polygon_with_regex, axis=1)
df_remaining = df_remaining[['Polygon']]
df_remaining.set_index('Polygon', inplace=True)
merged_df = df.join(df_remaining, on='Polygon')


                                              FILENAME  \
0    R01951-038.PORTSMOUTH RT 7 LCP #3A.(PR7LCP3AF)...   
1    VIRGINIA BEACH RT 30 LCP 5 R01952-091#5.(2205C...   
2    VIRGINIA BEACH RT 35 LCP 2  R01952-115#2.(2468...   
3    R01952-041.VIRGINIA BEACH RT 13 LCP #5 CAB 1.(...   
4    VIRGINIA BEACH RT 30 LCP 1 R01952-087#1.(2320S...   
..                                                 ...   
499  UNVERIFIED ADDRESS LIST - WINSTON SALEM 53 CAB...   
500  UNVERIFIED ADDRESS LIST - R01810-014 - E019DR2...   
501  UNVERIFIED ADDRESS LIST - R01810-006 - E026DR2...   
502      TROLLINGER ST (E032RM22) UNVERIFIED ADDRESSES   
503  UNVERIFIED ADDRESS LIST - R01810-001 - E013JT2...   

                               Polygon Project Number Work Order  
0              PORTSMOUTH RT 7 LCP #3A     R01951-038       None  
1           VIRGINIA BEACH RT 30 LCP 5     R01952-091       None  
2           VIRGINIA BEACH RT 35 LCP 2     R01952-115       None  
3    VIRGINIA BEACH RT 13 LCP #5 CA

In [ ]:
# Create a copy of the DataFrame 'df'.
#df_copy = df.copy()

# Keep only the rows in the copied DataFrame 'df_copy' where the 'Polygon' column is None.
#df_copy = df_copy[df_copy['Polygon'].isnull()]

# Reset the index of 'df_copy' to ensure a consistent index order.
#df_copy.reset_index(drop=True, inplace=True)

# Check the modified DataFrame 'df_copy' with rows filtered.
#print(df_copy)

In [27]:
#print(df)

merged_df.to_csv('Pre Dig FileName Xref 20230807 output.csv', index=False)
#complete_values.to_csv('PolygonParser_Complete.csv', index=False)